In [ ]:
! pip install datasets transformers rouge-score nltk py7zr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.5 MB/s 
     |████████████████████████████████| 5.8 MB 64.5 MB/s 
     |████████████████████████████████| 65 kB 5.0 MB/s 
     |████████████████████████████████| 182 kB 78.5 MB/s 
     |████████████████████████████████| 212 kB 69.5 MB/s 
     |████████████████████████████████| 132 kB 76.8 MB/s 
     |████████████████████████████████| 127 kB 80.1 MB/s 
     |████████████████████████████████| 7.6 MB 51.9 MB/s 
     |████████████████████████████████| 139 kB 58.1 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 50 kB 7.8 MB/s 
     |████████████████████████████████| 2.3 MB 67.1 MB/s 
     |████████████████████████████████| 357 kB 82.0 MB/s 
     |████████████████████████████████| 378 kB 83.5 MB/s 
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=8

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/NLP Project

/content/drive/MyDrive/NLP Project


# Fine-tuning a model on a summarization task

## Loading the dataset

In [ ]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("samsum")
metric = load_metric("rouge")

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

<ipython-input-4-a2ef1525156b>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


## BART

### Preprocessing the data

In [ ]:
# model_checkpoint_500 = "/content/drive/MyDrive/NLP Project/t5_results/checkpoint-500"
# model_checkpoint_1000 = "/content/drive/MyDrive/NLP Project/t5_results/checkpoint-1000"
# model_checkpoint_1500 = "/content/drive/MyDrive/NLP Project/t5_results/checkpoint-1500"
# model_checkpoint_2000 = "/content/drive/MyDrive/NLP Project/t5_results/checkpoint-2000"
# model_checkpoint_2500 = "/content/drive/MyDrive/NLP Project/t5_results/checkpoint-2500"
# model_checkpoint_3000 = "/content/drive/MyDrive/NLP Project/t5_results/checkpoint-3000"
# model_checkpoint_3500 = "/content/drive/MyDrive/NLP Project/t5_results/checkpoint-3500"
# model_checkpoint_4000 = "/content/drive/MyDrive/NLP Project/t5_results/checkpoint-4000"
model_checkpoint_2000 = "/content/drive/MyDrive/NLP Project with SCL/test-global-SCL-batch16/checkpoint-6000"



In [ ]:
from transformers import AutoTokenizer
    
# tokenizer_500 = AutoTokenizer.from_pretrained(model_checkpoint_500)
# tokenizer_1000 = AutoTokenizer.from_pretrained(model_checkpoint_1000)
# tokenizer_1500 = AutoTokenizer.from_pretrained(model_checkpoint_1500)
# tokenizer_2000 = AutoTokenizer.from_pretrained(model_checkpoint_2000)
# tokenizer_2500 = AutoTokenizer.from_pretrained(model_checkpoint_2500)
# tokenizer_3000 = AutoTokenizer.from_pretrained(model_checkpoint_3000)
# tokenizer_3500 = AutoTokenizer.from_pretrained(model_checkpoint_3500)
# tokenizer_4000 = AutoTokenizer.from_pretrained(model_checkpoint_4000)
tokenizer_2000 = AutoTokenizer.from_pretrained(model_checkpoint_2000)

loading file vocab.json
loading file merges.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json


In [ ]:
# string = "Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)"
# # print(tokenizer_500(["Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)"]))
# # print(tokenizer_500(["Amanda: I baked  cookies. Do you want some?"]))
# # print(tokenizer_500(["Jerry: Sure!"]))
# #token value is only effected by space: ""
# token = tokenizer_500(["Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)"])
# token_ids = token["input_ids"][0]
# print(token_ids)
# utterance_speaker_relation = []
# #[BOS] = 0 and [EOS] = 2
# #\r = 50121 , \n= 50118

# utt = []
# window = []
# for i in range(0,len(token_ids)):
#   if token_ids[i] == 2:
#     print("sentence ended")
#     utt.append(window)
#     break;
#   if token_ids[i]==0:
#     print("Sentence started")
#     continue;
#   window.append(token_ids[i])
#   if token_ids[i]==50118:
#     window = window[1:]
    
#   if token_ids[i]==50121:
#     # print(window[:-1])
#     utt.append(window[:-1])
#     window = []
#     i = i + 1

# for i in utt:
#   for j in range(0,len(i)):
#     if i[j]==35:
#       utterance_speaker_relation.append([i[:j],i[j+1:]])
#       break;

# print(utterance_speaker_relation)
# # print(tokenizer_500(["\r\n"]))




In [ ]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples["dialogue"]]
    model_inputs = tokenizer_2000(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer_2000.as_target_tokenizer():
        labels = tokenizer_2000(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/15 [00:00<?, ?ba/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# tokenized_datasets_train = tokenized_datasets['train'].select(range(100))
# tokenized_datasets_val = tokenized_datasets['validation'].select(range(70))

tokenized_datasets_train = tokenized_datasets['train']
tokenized_datasets_val = tokenized_datasets['validation']
# tokenized_datasets_test = tokenized_datasets['test']

### Fine-tuning the model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers.modeling_utils import unwrap_model
from transformers.models.auto.modeling_auto import MODEL_FOR_CAUSAL_LM_MAPPING_NAMES

model1 = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint_2000)
# model2 = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint_1000)
# model3 = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint_1500)
# model4 = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint_2000)


loading configuration file /content/drive/MyDrive/NLP Project with SCL/test-global-SCL-batch16/checkpoint-6000/config.json
Model config BartConfig {
  "_name_or_path": "/content/drive/MyDrive/NLP Project with SCL/test-global-SCL-batch16/checkpoint-6000",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartWithSCL"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "L

In [ ]:
batch_size = 6
args = Seq2SeqTrainingArguments(
    "test-dialogue-summarization",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    # save_total_limit=10,
    num_train_epochs=5,
    logging_steps = 10, ## added
    predict_with_generate=True,
    fp16=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# data_collator1 = DataCollatorForSeq2Seq(tokenizer_500, model=model1)
# data_collator2 = DataCollatorForSeq2Seq(tokenizer_1000, model=model2)
data_collator1 = DataCollatorForSeq2Seq(tokenizer_2000, model=model1)
# data_collator4 = DataCollatorForSeq2Seq(tokenizer_2000, model=model4)


In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer_2000.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer_2000.pad_token_id)
    decoded_labels =tokenizer_2000.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds] #my predictions from model
    
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]#original prediction

    
    for i in range(0,50):
      # print(tokenized_datasets_val["dialogue"][i])
      print("------------",i,"-------------")
      
      print("------>Predictions by Model")
      print(decoded_preds[i])
      print("----->Predictions Original")
      print(decoded_labels[i])
      print("**************************")
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer_2000.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
import torch.nn as nn
class CustomTrainer(Seq2SeqTrainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # How the loss is computed by Trainer. By default, all models return the loss in the first element.
        # Subclass and override for custom behavior.
      
        if self.label_smoother is not None and "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # print("outputs**********************: \n", outputs["last_hidden_state"].size())
        # print("logits*****************:", logits.view(-1, self.model.config.num_labels))
        # Save past state if it exists
        # TODO: this needs to be fixed and made cleaner later.
        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        if labels is not None:
            if unwrap_model(model)._get_name() in MODEL_FOR_CAUSAL_LM_MAPPING_NAMES.values():
                loss = self.label_smoother(outputs, labels, shift_labels=True)
            else:
                loss = self.label_smoother(outputs, labels)
            loss_fct = nn.CrossEntropyLoss()
            cross_loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
            loss += cross_loss
        else:
            if isinstance(outputs, dict) and "loss" not in outputs:
                raise ValueError(
                    "The model did not return a loss from the inputs, only the following keys: "
                    f"{','.join(outputs.keys())}. For reference, the inputs it received are {','.join(inputs.keys())}."
                )
            # We don't use .loss here since the model may return tuples instead of ModelOutput.
            loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]
        # Adding CrossEntropyLoss()
        # loss_fct = nn.CrossEntropyLoss()
        # cross_loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        # loss += cross_loss
        return (loss, outputs) if return_outputs else loss

In [ ]:
# trainer1 = CustomTrainer(
#     model1,
#     args,
#     train_dataset=tokenized_datasets_train,
#     eval_dataset=tokenized_datasets_test,
#     data_collator=data_collator1,
#     tokenizer=tokenizer_1500,
#     compute_metrics=compute_metrics
# )
# trainer2 = CustomTrainer(
#     model1,
#     args,
#     train_dataset=tokenized_datasets_train,
#     eval_dataset=tokenized_datasets_test,
#     data_collator=data_collator2,
#     tokenizer=tokenizer_1500,
#     compute_metrics=compute_metrics
# )
trainer3 = CustomTrainer(
    model1,
    args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_val,
    data_collator=data_collator1,
    tokenizer=tokenizer_2000,
    compute_metrics=compute_metrics
)
# trainer4 = CustomTrainer(
#     model4,
#     args,
#     train_dataset=tokenized_datasets_train,
#     eval_dataset=tokenized_datasets_test,
#     data_collator=data_collator4,
#     tokenizer=tokenizer_2000,
#     compute_metrics=compute_metrics
# )

# trainer = Seq2SeqTrainer(
#     model,
#     args,
#     train_dataset=tokenized_datasets_train,
#     eval_dataset=tokenized_datasets_val,
#     data_collator=data_collator,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics
# )

Using cuda_amp half precision backend


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
print(tokenized_datasets_val[8]['dialogue'])

Jane: Hello
Vegano Resto: Hello, how may I help you today?
Jane: I would like to make a reservation.
Jane: For 6 people, tonight around 20:00
Vegano Resto: Let me just check.
Vegano Resto: Ah, I'm afraid that there is no room at 20:00.
Vegano Resto: However, I could offer you a table for six at 18:30 or at 21:00
Vegano Resto: Would either of those times suit you?
Jane: Oh dear.
Jane: Let me just ask my friends.
Vegano Resto: No problem.
Jane: 21:00 will be ok.
Vegano Resto: Perfect. So tonight at 21:00 for six people under your name.
Jane: great, thank you!


In [ ]:
trainer3.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: dialogue, id, summary. If dialogue, id, summary are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 818
  Batch size = 32
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


------------ 0 -------------
------>Predictions by Model
A wants to get a puppy for her son.
She took him to the animal shelter
----->Predictions Original
A will go to the animal shelter tomorrow to get a puppy for her son.
They already visited the shelter last Monday and the son chose the puppy.
**************************
------------ 1 -------------
------>Predictions by Model
Emma wants to buy an advent calendar for her kids.
Rob used to get one
----->Predictions Original
Emma and Rob love the advent calendar.
Lauren fits inside calendar various items, for instance, small toys and Christmas decorations.
Her children are excited whenever they get the calendar.
**************************
------------ 2 -------------
------>Predictions by Model
Madison is pregnant.
Madison doesn't want to talk about it because she's worried about
----->Predictions Original
Madison is pregnant but she doesn't want to talk about it.
Patricia Stevens got married and she thought she was pregnant.
*********

{'eval_loss': 1.5140641927719116,
 'eval_rouge1': 47.9361,
 'eval_rouge2': 24.9374,
 'eval_rougeL': 40.6032,
 'eval_rougeLsum': 44.3042,
 'eval_gen_len': 18.2286,
 'eval_runtime': 45.5356,
 'eval_samples_per_second': 17.964,
 'eval_steps_per_second': 0.571}

In [ ]:
trainer4.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: id, summary, dialogue. If id, summary, dialogue are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 819
  Batch size = 32
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
-------------------------
------>Predictions by Model
Hannah doesn't know Betty's number.
She texted Larry last time they were at
----->Predictions Original
Hannah needs Betty's number but Amanda doesn't have it.
She needs to contact Larry.
**************************
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his 

{'eval_loss': 1.5441806316375732,
 'eval_rouge1': 46.407,
 'eval_rouge2': 22.8973,
 'eval_rougeL': 39.098,
 'eval_rougeLsum': 42.3675,
 'eval_gen_len': 18.2759,
 'eval_runtime': 58.765,
 'eval_samples_per_second': 13.937,
 'eval_steps_per_second': 0.442}

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, dialogue, id. If summary, dialogue, id are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 818
  Batch size = 32


{'eval_loss': 1.5303208827972412,
 'eval_rouge1': 47.5105,
 'eval_rouge2': 24.1888,
 'eval_rougeL': 40.0868,
 'eval_rougeLsum': 43.8536,
 'eval_gen_len': 18.1125,
 'eval_runtime': 44.2729,
 'eval_samples_per_second': 18.476,
 'eval_steps_per_second': 0.587,
 'epoch': 5.0}

## T5

### Preprocessing the data

In [ ]:
model_checkpoint_500 = "/content/drive/MyDrive/NLP Project/t5_results/checkpoint-500"
model_checkpoint_1000 = "/content/drive/MyDrive/NLP Project/t5_results/checkpoint-1000"
model_checkpoint_1500 = "/content/drive/MyDrive/NLP Project/t5_results/checkpoint-1500"
model_checkpoint_2000 = "/content/drive/MyDrive/NLP Project/t5_results/checkpoint-2000"
model_checkpoint_2500 = "/content/drive/MyDrive/NLP Project/t5_results/checkpoint-2500"
model_checkpoint_3000 = "/content/drive/MyDrive/NLP Project/t5_results/checkpoint-3000"
model_checkpoint_3500 = "/content/drive/MyDrive/NLP Project/t5_results/checkpoint-3500"
model_checkpoint_4000 = "/content/drive/MyDrive/NLP Project/t5_results/checkpoint-4000"
model_checkpoint_4500 = "/content/drive/MyDrive/NLP Project/t5_results/checkpoint-4500"



In [ ]:
from transformers import AutoTokenizer
    
tokenizer_500 = AutoTokenizer.from_pretrained(model_checkpoint_500)
tokenizer_1000 = AutoTokenizer.from_pretrained(model_checkpoint_1000)
tokenizer_1500 = AutoTokenizer.from_pretrained(model_checkpoint_1500)
tokenizer_2000 = AutoTokenizer.from_pretrained(model_checkpoint_2000)
# tokenizer_2500 = AutoTokenizer.from_pretrained(model_checkpoint_2500)
# tokenizer_3000 = AutoTokenizer.from_pretrained(model_checkpoint_3000)
# tokenizer_3500 = AutoTokenizer.from_pretrained(model_checkpoint_3500)
# tokenizer_4000 = AutoTokenizer.from_pretrained(model_checkpoint_4000)
# tokenizer_4500 = AutoTokenizer.from_pretrained(model_checkpoint_4500)

In [ ]:
model_checkpoint = "t5-base"

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
    
model_500 = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint_500)
model_1000 = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint_1000)
model_1500 = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint_1500)
model_2000 = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint_2000)
# model_2500 = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint_2500)
# model_3000 = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint_3000)
# model_3500 = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint_3500)
# model_4000 = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint_4000)
# model_4500 = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint_4500)



In [ ]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    task_prefix = "summarize: "
    inputs = examples["dialogue"]
    model_inputs = tokenizer_500([task_prefix + dialogue for dialogue in inputs], 
                             padding="max_length",
                             max_length=max_input_length, 
                             truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer_500(examples["summary"], 
                        padding="max_length",
                        max_length=max_target_length, 
                        truncation=True)

    model_inputs["labels"] = [
        [-100 if token == tokenizer_500.pad_token_id else token for token in l]
        for l in labels["input_ids"]]
        
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# sample a small set for development
# tokenized_datasets_train = tokenized_datasets['train'].select(range(100))
# tokenized_datasets_val = tokenized_datasets['validation'].select(range(70))


tokenized_datasets_train = tokenized_datasets['train']
tokenized_datasets_test = tokenized_datasets['test']
tokenized_datasets_val = tokenized_datasets['validation']

In [ ]:
for i in range(0,5):
      print(tokenized_datasets_val["dialogue"][i])
      print("***********************")

A: Hi Tom, are you busy tomorrow’s afternoon?
B: I’m pretty sure I am. What’s up?
A: Can you go with me to the animal shelter?.
B: What do you want to do?
A: I want to get a puppy for my son.
B: That will make him so happy.
A: Yeah, we’ve discussed it many times. I think he’s ready now.
B: That’s good. Raising a dog is a tough issue. Like having a baby ;-) 
A: I'll get him one of those little dogs.
B: One that won't grow up too big;-)
A: And eat too much;-))
B: Do you know which one he would like?
A: Oh, yes, I took him there last Monday. He showed me one that he really liked.
B: I bet you had to drag him away.
A: He wanted to take it home right away ;-).
B: I wonder what he'll name it.
A: He said he’d name it after his dead hamster – Lemmy  - he's  a great Motorhead fan :-)))
***********************
Emma: I’ve just fallen in love with this advent calendar! Awesome! I wanna one for my kids!
Rob: I used to get one every year as a child! Loved them! 
Emma: Yeah, i remember! they were fil

### Fine-tuning the model

In [ ]:
# Parameters\
batch_size=8
training_args = Seq2SeqTrainingArguments(
    output_dir="t5_results",
    num_train_epochs=5,
    do_train=True,
    do_eval=True,
    evaluation_strategy = "epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=1e-4,
    warmup_steps=500,
    weight_decay=0.1,
    # label_smoothing_factor=0.1, ## causes to throw an error
    predict_with_generate=True,
    # logging_dir="logs",
    logging_steps=10
)


data_collator = DataCollatorForSeq2Seq(tokenizer_500, model=model_500)
data_collator = DataCollatorForSeq2Seq(tokenizer_1000, model=model_1000)
data_collator = DataCollatorForSeq2Seq(tokenizer_500, model=model_1500)
data_collator = DataCollatorForSeq2Seq(tokenizer_1000, model=model_2000)
data_collator = DataCollatorForSeq2Seq(tokenizer_500, model=model_2500)
data_collator = DataCollatorForSeq2Seq(tokenizer_1000, model=model_3000)
data_collator = DataCollatorForSeq2Seq(tokenizer_500, model=model_3500)
data_collator = DataCollatorForSeq2Seq(tokenizer_500, model=model_4000)
data_collator = DataCollatorForSeq2Seq(tokenizer_500, model=model_4500)

trainer1 = CustomTrainer(
    model=model_500,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_test,
    tokenizer=tokenizer_500,
    compute_metrics=compute_metrics,
)
trainer2 = CustomTrainer(
    model=model_1000,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_test,
    tokenizer=tokenizer_500,
    compute_metrics=compute_metrics,
)
trainer3 = CustomTrainer(
    model=model_1500,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_test,
    tokenizer=tokenizer_500,
    compute_metrics=compute_metrics,
)
trainer4 = CustomTrainer(
    model=model_2000,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_test,
    tokenizer=tokenizer_500,
    compute_metrics=compute_metrics,
)
trainer5 = CustomTrainer(
    model=model_2500,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_test,
    tokenizer=tokenizer_500,
    compute_metrics=compute_metrics,
)
trainer6 = CustomTrainer(
    model=model_3000,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_test,
    tokenizer=tokenizer_500,
    compute_metrics=compute_metrics,
)
trainer7 = CustomTrainer(
    model=model_3500,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_test,
    tokenizer=tokenizer_500,
    compute_metrics=compute_metrics,
)
trainer8 = CustomTrainer(
    model=model_4000,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_test,
    tokenizer=tokenizer_500,
    compute_metrics=compute_metrics,
)
trainer9 = CustomTrainer(
    model=model_3500,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_test,
    tokenizer=tokenizer_500,
    compute_metrics=compute_metrics,
)

In [ ]:
# evaluate before training for comparison
trainer1.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: dialogue, summary, id. If dialogue, summary, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 818
  Batch size = 8


Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
-------------------------
------>Predictions by Model
A wants to get a puppy for his son.
He's ready now.
B will
----->Predictions Original
A will go to the animal shelter tomorrow to get a puppy for her son.
They already visited the shelter last Monday and the son chose the puppy.
**************************
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.

{'eval_loss': 1.443723201751709,
 'eval_rouge1': 47.0787,
 'eval_rouge2': 23.734,
 'eval_rougeL': 40.0132,
 'eval_rougeLsum': 43.461,
 'eval_gen_len': 16.9328,
 'eval_runtime': 112.7673,
 'eval_samples_per_second': 7.254,
 'eval_steps_per_second': 0.913}

In [ ]:
trainer2.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: dialogue, summary, id. If dialogue, summary, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 818
  Batch size = 8


Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
-------------------------
------>Predictions by Model
A wants to get a puppy for his son.
He's already discussed it many times
----->Predictions Original
A will go to the animal shelter tomorrow to get a puppy for her son.
They already visited the shelter last Monday and the son chose the puppy.
**************************
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll chec

{'eval_loss': 1.4845283031463623,
 'eval_rouge1': 47.1742,
 'eval_rouge2': 23.645,
 'eval_rougeL': 39.3825,
 'eval_rougeLsum': 43.0994,
 'eval_gen_len': 16.945,
 'eval_runtime': 113.1148,
 'eval_samples_per_second': 7.232,
 'eval_steps_per_second': 0.911}

In [ ]:
trainer3.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: dialogue, summary, id. If dialogue, summary, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 818
  Batch size = 8


Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
-------------------------
------>Predictions by Model
A wants to get a puppy for his son.
B will take him to the animal shelter
----->Predictions Original
A will go to the animal shelter tomorrow to get a puppy for her son.
They already visited the shelter last Monday and the son chose the puppy.
**************************
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll che

{'eval_loss': 1.4714404344558716,
 'eval_rouge1': 47.5949,
 'eval_rouge2': 24.2,
 'eval_rougeL': 39.8008,
 'eval_rougeLsum': 43.8098,
 'eval_gen_len': 17.3814,
 'eval_runtime': 119.3292,
 'eval_samples_per_second': 6.855,
 'eval_steps_per_second': 0.863}

In [ ]:
trainer4.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: dialogue, summary, id. If dialogue, summary, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 818
  Batch size = 8


Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
-------------------------
------>Predictions by Model
A wants to get a puppy for his son.
He wants one that won't grow
----->Predictions Original
A will go to the animal shelter tomorrow to get a puppy for her son.
They already visited the shelter last Monday and the son chose the puppy.
**************************
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric:

{'eval_loss': 1.4294675588607788,
 'eval_rouge1': 47.7572,
 'eval_rouge2': 23.9704,
 'eval_rougeL': 39.9791,
 'eval_rougeLsum': 43.7941,
 'eval_gen_len': 17.0844,
 'eval_runtime': 119.4155,
 'eval_samples_per_second': 6.85,
 'eval_steps_per_second': 0.863}

In [ ]:
trainer5.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: dialogue, summary, id. If dialogue, summary, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 818
  Batch size = 8


Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
-------------------------
------>Predictions by Model
A wants to get a puppy for his son.
B will take him to the animal shelter
----->Predictions Original
A will go to the animal shelter tomorrow to get a puppy for her son.
They already visited the shelter last Monday and the son chose the puppy.
**************************
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll che

{'eval_loss': 1.4158828258514404,
 'eval_rouge1': 48.3651,
 'eval_rouge2': 24.466,
 'eval_rougeL': 40.3832,
 'eval_rougeLsum': 44.4183,
 'eval_gen_len': 17.2408,
 'eval_runtime': 119.8599,
 'eval_samples_per_second': 6.825,
 'eval_steps_per_second': 0.859}

In [ ]:
trainer6.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: dialogue, summary, id. If dialogue, summary, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 818
  Batch size = 8


Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
-------------------------
------>Predictions by Model
A wants to get a puppy for his son.
He will name it after his dead
----->Predictions Original
A will go to the animal shelter tomorrow to get a puppy for her son.
They already visited the shelter last Monday and the son chose the puppy.
**************************
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eri

{'eval_loss': 1.4026352167129517,
 'eval_rouge1': 47.915,
 'eval_rouge2': 24.7222,
 'eval_rougeL': 40.7964,
 'eval_rougeLsum': 44.2571,
 'eval_gen_len': 16.7995,
 'eval_runtime': 118.8484,
 'eval_samples_per_second': 6.883,
 'eval_steps_per_second': 0.867}

In [ ]:
trainer7.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: dialogue, summary, id. If dialogue, summary, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 818
  Batch size = 8


Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
-------------------------
------>Predictions by Model
A wants to get a puppy for his son.
He will take it home.
----->Predictions Original
A will go to the animal shelter tomorrow to get a puppy for her son.
They already visited the shelter last Monday and the son chose the puppy.
**************************
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.


{'eval_loss': 1.3907883167266846,
 'eval_rouge1': 47.7305,
 'eval_rouge2': 24.1803,
 'eval_rougeL': 39.7421,
 'eval_rougeLsum': 43.7305,
 'eval_gen_len': 16.8606,
 'eval_runtime': 119.409,
 'eval_samples_per_second': 6.85,
 'eval_steps_per_second': 0.863}

In [ ]:
trainer8.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: dialogue, summary, id. If dialogue, summary, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 818
  Batch size = 8


Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
-------------------------
------>Predictions by Model
A wants to get a puppy for his son.
B will take him to the animal shelter
----->Predictions Original
A will go to the animal shelter tomorrow to get a puppy for her son.
They already visited the shelter last Monday and the son chose the puppy.
**************************
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll che

{'eval_loss': 1.409375786781311,
 'eval_rouge1': 48.0171,
 'eval_rouge2': 24.3597,
 'eval_rougeL': 40.2897,
 'eval_rougeLsum': 44.2865,
 'eval_gen_len': 17.1467,
 'eval_runtime': 118.6227,
 'eval_samples_per_second': 6.896,
 'eval_steps_per_second': 0.868}

In [ ]:
trainer9.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: dialogue, summary, id. If dialogue, summary, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 818
  Batch size = 8


Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
-------------------------
------>Predictions by Model
A wants to get a puppy for his son.
He will take it home.
----->Predictions Original
A will go to the animal shelter tomorrow to get a puppy for her son.
They already visited the shelter last Monday and the son chose the puppy.
**************************
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.


{'eval_loss': 1.3907883167266846,
 'eval_rouge1': 47.7305,
 'eval_rouge2': 24.1803,
 'eval_rougeL': 39.7421,
 'eval_rougeLsum': 43.7305,
 'eval_gen_len': 16.8606,
 'eval_runtime': 119.4043,
 'eval_samples_per_second': 6.851,
 'eval_steps_per_second': 0.863}

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: summary, dialogue, id. If summary, dialogue, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14732
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 9210
  Number of trainable parameters = 222903552


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.378600,1.459050,47.075900,23.546800,39.695500,43.446200,16.924200


Saving model checkpoint to t5_results/checkpoint-500
Configuration saved in t5_results/checkpoint-500/config.json
Model weights saved in t5_results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in t5_results/checkpoint-500/tokenizer_config.json
Special tokens file saved in t5_results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to t5_results/checkpoint-1000
Configuration saved in t5_results/checkpoint-1000/config.json
Model weights saved in t5_results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in t5_results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in t5_results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to t5_results/checkpoint-1500
Configuration saved in t5_results/checkpoint-1500/config.json
Model weights saved in t5_results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in t5_results/checkpoint-1500/tokenizer_config.json
Special tokens file saved in t5_results/checkpoint-15

Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
-------------------------
------>Predictions by Model
A wants to get a puppy for his son.
He wants one that won't grow
----->Predictions Original
A will go to the animal shelter tomorrow to get a puppy for her son.
They already visited the shelter last Monday and the son chose the puppy.
**************************
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric:

Saving model checkpoint to t5_results/checkpoint-2000
Configuration saved in t5_results/checkpoint-2000/config.json
Model weights saved in t5_results/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in t5_results/checkpoint-2000/tokenizer_config.json
Special tokens file saved in t5_results/checkpoint-2000/special_tokens_map.json
Saving model checkpoint to t5_results/checkpoint-2500
Configuration saved in t5_results/checkpoint-2500/config.json
Model weights saved in t5_results/checkpoint-2500/pytorch_model.bin
tokenizer config file saved in t5_results/checkpoint-2500/tokenizer_config.json
Special tokens file saved in t5_results/checkpoint-2500/special_tokens_map.json
Saving model checkpoint to t5_results/checkpoint-3000
Configuration saved in t5_results/checkpoint-3000/config.json
Model weights saved in t5_results/checkpoint-3000/pytorch_model.bin
tokenizer config file saved in t5_results/checkpoint-3000/tokenizer_config.json
Special tokens file saved in t5_results/checkpoi

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: summary, dialogue, id. If summary, dialogue, id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 818
  Batch size = 8


{'eval_loss': 1.4014945030212402,
 'eval_rouge1': 48.1292,
 'eval_rouge2': 24.7742,
 'eval_rougeL': 40.5648,
 'eval_rougeLsum': 44.3401,
 'eval_gen_len': 17.0868,
 'eval_runtime': 108.3146,
 'eval_samples_per_second': 7.552,
 'eval_steps_per_second': 0.951,
 'epoch': 5.0}